# Turbulence Closure Schemes in ICON-LEM

## 1. Motivation/Introduction
***

One of the main reasons for uncertainties of numerical models comes from parameterizations, especially those dealing with sub-grid processes. For large-eddy simulation models (LES or LEM) the effect of the parameterizations in the entrainment zone was shown to be of leading order. In this region processes like advection and diffusion of the subgrid-scale (SGS) turbulent kinetic energy (TKE) as well as the time-rate-of-change term, that can be seen as the memory of the flow, are particularly important and it is crucial to capture occuring SGS processes as precise as possible.

Stratocumulus-topped boundary layer (STBL).

## 2. Theory
***

When using LES to model atmospheric flows, by filtering the governing equations covariance terms appear. Since these terms contain unknown turbulence information from the 
subgrid-scale, they can not be solved numerically and therefore have to be parameterized. In the literature this 
problem is assosciated with different names like turbulence closure or subgrid-scale modelling. The most common approach for LES of the planetary boundary layer (PBL) is to link the energy transport by SGS motions, e.g. the flux of momentum $\overline{u_{i}'u_{j}'}$ or flux of some scalar $\overline{u_i'\psi'}$, to the mean flow gradient
$$\begin{align}
 \overline{u_{i}'u_{j}'} &= -\,K_\mathrm{m}\,\left(\frac{\partial u_i}{\partial x_j} + 
                                                   \frac{\partial u_j}{\partial x_i} \right) \text{   ,} 
                                                   \label{eq:momentumflux}
\tag{1}
\\
\overline{u_i'\psi'} &= -\,K_\mathrm{h}\,\frac{\partial \psi}{\partial x_i} \text{   .}
                                                    \label{eq:scalarflux}
\tag{2}
\end{align} $$
In this, $K_\mathrm{m}$ and $K_\mathrm{h}$ are the local SGS eddy diffusivities for momentum and heat, $\psi$ could be the temperature, moisture, liquid water content or any passive scalar. The
diffusivities have a direct relationship to the SGS-TKE ($e = \frac{1}{2}\,\overline{u_i'u_i'}$) and therefore can be derived
from it. To do this, different approaches exist. In the following the already in ICON-LEM implemented 
Smagorinsky model (INSERT CITATION!!!) will be explained as well as the 1.5-order turbulence closure follwing
Deardorff (INSERT CITATION!!!). <br>
Both ideas are based on the prognostic equation for the SGS-TKE $e$ that reads
$$\begin{equation}
\frac{\partial e}{\partial t} = \frac{1}{\rho}\frac{\partial(\rho\,\overline{u}_i\, e)}{\partial x_j} - \overline{(u_i' u_j')}\frac{\partial \overline{u}_i}{\partial x_j} + \frac{g}{\theta_\mathrm{v,0}} \overline{u_3'\theta'_\mathrm{v}} - \frac{1}{\rho} \frac{\partial}{\partial x_i} \left(\overline{\rho\,u'_i(e'+\frac{p'}{\rho})}\right) - \epsilon \text{   .}                                                    \label{eq:fullTKE}
\tag{3}
\end{equation}$$
From left to right the terms are the time-rate of change, the advection of TKE, the production of TKE by shear, by buoyancy, the diffusive transport of TKE and the diffusion $\epsilon$.  <br>
How the different closure schemes work will be further described in the following sections.

### 2.1 The Smagorinsky model

The Smagorinsky approach is based on the local equilibrium constraint that implies a balance of production of TKE 
through shear and buoyancy with the dissipation of TKE. With this assumption the prognostic equation simplifies to
\begin{equation}
\overline{(u_i' u_j')}\frac{\partial \overline{u}_i}{\partial x_j} + \frac{g}{\theta_\mathrm{v,0}}\overline{u_3'\theta'_\mathrm{v}} = \epsilon \text{   .}
                                                   \label{eq:smago}
\tag{4}
\end{equation}
Using the subgrid-scale stress tensor and its parameterization accoring Lilly (INSERT CITATION!!!) the subgrid diffusivity for momentum can be written as
\begin{align}
K_\mathrm{m} &= 2\,\lambda^2\rho\,||S||\,\left(1 - \frac{\mathrm{Ri}}{\mathrm{Pr}_\mathrm{t}} \right)^{1/2}
                                                   \label{eq:smagodiffmom}
\tag{5}
\qquad \text{for} \qquad
\frac{\mathrm{Ri}}{\mathrm{Pr}_\mathrm{t}} > 0 \text{   ,} \\
K_\mathrm{h} &= \frac{K_\mathrm{m}}{\mathrm{Pr}_\mathrm{t}} \text{   .}
                                                   \label{eq:smagodiffheat}
\tag{6}
\end{align}
Here, $\lambda$ is the subgrid length scale for the Smagorinksy approach, which is proportional to the grid volume amongst other factors. $\rho$ describes the density, $||S|| = \sqrt{S^2} = (S_\mathrm{ij}S_\mathrm{ij})^{1/2}$ is the deformation through shear, $\mathrm{Ri}$ the Richardson and $\mathrm{Pr}_\mathrm{t}$ the turbulent Prandl number. 
<br>
Because the Smagorinsky model is the already implemented and closure scheme, additional details will be neglected at this point. For further general information about the Smagorinsky model the reader is referred to the work of Lilly (INSERT CITATION!!!) or for its implementation in the ICON-LEM to Dipankar (INSERT CITATION!!!).

### 2.2 The Deardorff model

To capture all of the important SGS processes the approach following Deardorff (INSERT CITATION!!!) that takes all of the terms of the prognostic equation for the SGS-TKE into consideration. This allows to tie the SGS eddy diffusivites directly to the SGS-TKE

\begin{align}
K_\mathrm{m} &= c_\mathrm{m}\, l \, \sqrt{e} \text{   ,} 
\label{eq:deardiffmom}
\tag{7}
\\
K_\mathrm{h} &= \left( 1 + 2\,\frac{l}{\Delta} \right)\, K_\mathrm{m} \text{   ,}
\label{eq:deardiffheat}
\tag{8}
\end{align}

with a model constant $c_\mathrm{m} = 0.1$ , the average grid spacing $\Delta = (\Delta x\Delta y\Delta z)^{1/3}$ being the composite of the grid spacing of the three directions $x$, $y$ and $z$ and the SGS mixing length $l$. The latter depends on the height above the ground limiting the diameter of eddies, as also on the stratification and the average gridspacing $\Delta$. It is calculated as

\begin{align}
l = 
  \begin{cases}
    \mathrm{min} \left(1.8\,z\,,\,\Delta\,, 0.76\sqrt{e}\,\left( \frac{g}{\theta_\mathrm{v}}\,\frac{\partial\,\theta\mathrm{v}}{\partial \,z}\right)^{-1/2} \right)&,  \text{for } \frac{\partial \,\theta\mathrm{v}}{\partial \,z} > 0 \\
   \mathrm{min} \,\bigg( 1.8\,z\,,\,\Delta \bigg)&,  \text{for } \frac{\partial \,\theta\mathrm{v}}{\partial\, z} \leq 0 \\
\label{eq:mixinglength}
\tag{9}
  \end{cases}
\end{align}

In this work the modified version of the originial prognostic equation by Moeng and Wyngaard and Saiki et al (INSERT CITATION!!!) is being used. The substitutions of the covariance terms read as follows

\begin{align}
\overline{(u_i' u_j')}\,\frac{\partial \overline{u}_i}{\partial x_j} &= -\,K_\mathrm{m}\,S^2 \text{   ,} 
\label{eq:shearterm}
\tag{10} \\
\frac{g}{\theta_\mathrm{v,0}}\,\overline{u_3'\theta'_\mathrm{v}} &= -\,K_\mathrm{h}\,N^2 \text{   ,} 
\label{eq:buoyterm}
\tag{11} \\
\end{align}

with the deformation $S^2$ as in the Smagorinsky model and the squared Brunt-Vaisala frequency $N^2$. The pressure term is parameterized as

\begin{align}
\frac{1}{\rho}\frac{\partial}{\partial x_i} \left(\overline{\rho\,u_i'(e'+\frac{p'}{\rho})} \right ) &= - \frac{1}{\rho}\frac{\partial}{\partial x_i} \left( 2\,\rho\,K_\mathrm{m}\frac{\partial e}{\partial x_i} \right )
\label{eq:differm}
\tag{12}
\end{align}
and the dissipation $\epsilon$ is given by
\begin{align}
\epsilon = \left( 0.19 + 0.74\,\frac{l}{\Delta}  \right)\,\frac{e^{3/2}}{l}
\label{eq:dissipationterm}
\tag{13}
\end{align}

In the following chapter it will be shown how the now fully definied prognostic equation is technically implemented and who the investigated test cases look like.

## 3. Technical implementation
***

The implementation of most physical schemes demand special numerical treatment and therefor they can only rarely be converted from one model to another. For LES there are special challenges that have to be taken care for like the time-stepping, the method to calculate horizontal derivatives and the general grid structure, since these things vary quite frequently depending on the general assumptions made, the purpose the model serves for and the needed complexity. <br>
Due to it's embedding in the ICON framework, the ICON-LEM has some very unique features and needs. In the first part of this chapter the focus is exactly on these details, whereas the second part covers the setups for the validation of the Deardorff scheme.

### General

Understand the technical details of the implementation one has to start with the basic features of ICON and its LES that distinguishes it from most other numerical models. Two of its key components can be derived directly from the models full name **ICO**sahedral **N**onhydrostatic Model:  <br>

1. All equations are solved on an icosahedral grid,
2. The modell uses the fully compressible set of equations.

The icosahedral grid is of special importance because the definition of the horizontal directions ($x_\mathrm{1}, x_\mathrm{2}$) is not classical but the coordinates are oriented normal ($x_\mathrm{1}$) and parallel ($x_\mathrm{2}$) to the triangle edges. Together with the third coordinate ($x_\mathrm{3}$) following the terrain-following SLEVE coordinate system ($x_\mathrm{1}, x_\mathrm{2}, \overline{x}_\mathrm{3}$ ), they form a right-handed system. A second point to the mesh is the staggered position of variables in an Arakawa-C grid (INSERT CITATION!!!) of an icosahedral mesh. Both the definition of the wind vector as well as the staggering affect terms that include differentials with a special treatment of horizontal derivatives. The following sketch originally stems from Wan (INSER CITATION) and depicts the triangular grid structure with full and intermediate levels and the positioning of the different variables. In ICON, the numbering of the vertical level is reversed, so that the model top is $k=0$ increasing towards the model surface.<br>
<br>
<figure><img src="files/grid_ICON.png" width="750" height="1500" align="left"><figcaption><br>* Fig.1: Illustration of the triangular grid and the location of examplary
variables.*</figcaption></figure> <br>
<br>
Just by looking at these main variables it can easily be seen that they are at different location compared to a classical rectangular grid. While the majority of the scalars (like the temperature $T$) is defined at full level cell centers, others like the pressure $p$ live on the interface (also called half) levels, being shifted by half a grid spacing in the vertical. The aformentioned wind speed $v_\mathrm{n}$ is defined in the middle of the edges on full levels, while the potential vorticity $\zeta$ can be found at the vertices of each triangle. Combining the positioning and the general triangular shape this leads to a different total number of grid points for each variable position, what has to be considered when constructing derivatives e.g. for the advection or flux calculations. The SGS-TKE is defined at the same point where the eddy diffusivities are living, since interpolating the latter can have severe impact on the flow. Exemplary for this, one could look at a STBL, that is characterized by two distinct formed layer near the cloud top with completly different flow quantities. When interpolating one grid point may lay inside the cloud while the other will be inside the inversion and therefor has very suppressed SGS movement. The interpolation between this point weakens the strong gradient of the eddy diffusivities and has the potential to smear the border between the distinct formed layers. 
<br>
The fact that the model is nonhydrostatic deserves mentioning since the variable density allows sound-wave propagation. To deal with the high propagation speed of these a time-splitting into substeps inside the dynamical core is required. The typical dynamical time step is therefore about 0.02 times smaller than those of typical LES models like PALM (Maronga et al.) or UCLA-LES (Stevens et al.). <br>

### Realization and Numerics

A brief description of the numerical implementation of the single terms of the prognostic equation for the SGS-TKE is important to understand the results of the new scheme and possible differences to other schemes or models (here: Smagorinsky and Deardorff in PALM). We will start on the right side of the equality sign with the advection term, going on from left to right. <br>
<br>
In high resolution simulations, the SGS-TKE should be small compared to the resolved part, so that it should have a comparebly short lifetime before beeing dissipated. Stating this, the SGS-TKE is treated as if it would be a passive scalar which does not interact with the flow itself. That said, the existing advection routines for passive scalars can be used to advect the SGS-TKE. We forgo on an in-depth explanation of the underlying equations and routines and refer to the paper of Zaengl et al. (INSER CITATION). One restriction that comes with this course of action is the actual model level used in the advection routines. While passive scalars are advected on full levels, the SGS-TKE is defined on interface levels. Therefore the advective tendencies have to be vertically interpolated to interface levels, resulting in an interpolation error, while the temporal integrated SGS-TKE has to be interpolated back to full levels. Inside chapter results a small section will focus on the error that comes with these interpolations and its dependence on the vertical spacing. <br>
<br>
The next two terms include the production of SGS-TKE through shear and buoyancy. Both terms are also part of the Smagorinsky scheme. Production through shear inside the Deardorff scheme is exactly as in the Smagorinsky scheme where:
$\begin{align}
S &= \sum_{i=1}^3 \sum_{j=1}^3 S_{ij} \text{  ,  } \\
\label{eq:deform}
\tag{14}
S_\mathrm{11} & = \frac{\partial u_\mathrm{1}}{\partial x_\mathrm{1}} \text{  ,  } \\
S_\mathrm{12} & = \frac{1}{2} \, \left( \frac{\partial u_\mathrm{1}}{\partial x_\mathrm{2}} + \frac{\partial u_\mathrm{2}}{\partial x_\mathrm{1}} \right) \text{  ,  } \tag{15}\\
S_\mathrm{13} & = \frac{1}{2} \, \left( \frac{\partial u_\mathrm{1}}{\partial x_\mathrm{3}} + \frac{\partial u_\mathrm{3}}{\partial x_\mathrm{1}} \right) \text{  ,  } \tag{16}\\
S_\mathrm{22} & = \frac{\partial u_\mathrm{2}}{\partial x_\mathrm{2}} \text{  ,  } \tag{17}\\
S_\mathrm{21} & = S_\mathrm{12} \text{  ,  } \tag{18}\\
S_\mathrm{23} & = \frac{1}{2} \, \left( \frac{\partial u_\mathrm{2}}{\partial x_\mathrm{3}} + \frac{\partial u_\mathrm{3}}{\partial x_\mathrm{2}} \right) \text{  ,  } \tag{19}\\
S_\mathrm{33} & = \frac{\partial u_\mathrm{3}}{\partial x_\mathrm{3}} \text{  ,  } \tag{20}\\
S_\mathrm{31} & = S_\mathrm{13} \text{  ,  } \tag{21}\\
S_\mathrm{32} & = S_\mathrm{23} \text{  .  } \tag{22}
\end{align} $<br>
Here $u_i$ is the velocity component and $x_i$ is  and the regarding coordinate.
For the production through buoyancy we use the Brunt-Vaisala frequency $N^2$, that is defined as
\begin{equation}
N^2 = \frac{g}{\theta_\mathrm{v,0}}\,\frac{\partial \theta_\mathrm{v}}{\partial z} \text{  .  } 
\label{eq:bruntvais}
\tag{23}
\end{equation}
The partial differential is approached via finite differences, but due to numerical stability the difference should be calculated over to grid boxes
\begin{equation}
\frac{\partial \theta_\mathrm{v}}{\partial z} = \frac{\left[\theta_\mathrm{v}(k-1)-\theta_\mathrm{v}(k+1)\right]}{[\Delta(k-1)+\Delta(k)]} \text{  .  }
\label{eq:vertgrad}
\tag{24}
\end{equation}
The fourth term inhibits third-moments and is modeled diffusively as shown above (equation 12). Inside the model we added another diffusion subroutine for the SGS-TKE since the existing one for temperature and moisture operates again on full levels. During the implementation it showed that interpolating the diffusive tendency, when calculated on full levels, causes $2\Delta x$ waves in the vertical direction due to the strong gradient  of $e$ near the surface. <br>
The remaining term is the dissipation of SGS-TKE which also exists in the Smagorinsky scheme. Nevertheless these terms are not equivalent since the calculation in the Smagorinsky scheme is based on the local equilibrium limit and therefor is exactly the sum of the production terms, while in the Deardorff scheme the dissipation is linked to the local value of the SGS-TKE.


### Boundary Conditions

While the influence of the upper boundary condition for the SGS-TKE should be of low relevance, because the model top should be outside of the PBL where the importance of the SGS motions is negligible, the lower boundary condition is a crucial question. Since the SGS-TKE lives on intermediate levels the lowest grid point is slightly above the surface, where the importance of the SGS processes is several orders higher than that of the resolved motions, it should have its maximum. On the other hand, the SGS-TKE should be zero exactly at the surface, resulting in a very strong vertical gradient between the surface and the first grid point. A solution to keep the SGS-TKE in sync with the diffusivities, which are assumed constant keep flux calculations stable, a Neumann boundary condition is applied which also yields numerical stability.

### Metric Terms

Another important factor comes into play for all terms of equations that include horizontal derivatives. Due to the use of topography in real case simulations it is necessary to account for different elevations when using horizontal adjacent grid cells. One way to do so is to use metric correction terms. Those are discussed in the paper of Brdar et al. (INSERT CITATION!!!) and are getting applied in the Smagorinsky scheme as well as in all diffusion equations. 
<br>
<br> THINK IF THIS PART IS NECESSARY SINCE THE EFFECT OF THE METRIC TERMS MAY BE NEGLIGIBLE.

## 4. Setup
***

Following the work of Dipankar et al. (INSERT CITATION) setups for a dry and a cloud topped convective boundary layer case were chosen to test and compare the results of the turbulence schemes of ICON-LEM against eachother and also against simulations with PALM (**Pa**rallelilized **L**arge-Eddy **M**odel) from the Institute of Meteorology and Climatology at the Leibniz Universitaet Hannover (Maronga et al. INSERT CITATION!!!). Because the numerics used for the underlying grid of the ICON-LEM diverge from standard LES models like PALM differences in the simulation results can be expected, eventhough both models run the same model setup. A short list of core differences can also be found in Dipankar et al. (INSER CITATION) and will not be repeated at this point.

The model domain is a doubly periodic torus grid and has a size of 9.6 x 9.6 $\mathrm{km}^2$ in horizontal directions and a vertical extent of 3.2 km. The influence of the subgrid scale is defined by the grid spacing that determines the minimum diameter of resolved vortices. To quantify this, the horizontal grid spacing is varied $\Delta x = \Delta y = \{ 25\,\mathrm{m},100\,\mathrm{m} \} $. For a torus grid the effective resolution and the used grid spacing are geometrically not the same. It is therefore needed to use an equivalent triangle edge length $\Delta l$, obtained by equating the area of the cells as 
\begin{equation}
\Delta l \approx 1.5 \sqrt{\Delta x \Delta y}\approx \{ 37.5\,\mathrm{m}, 150\,\mathrm{m} \} \text{  .  }
\label{eq:gridspace}
\tag{25}
\end{equation} Further information on this can be found in Dipankar et al. (INSERT CITATION), where this problem is elaborated in more detail. In the beginning vertical spacing  of $\Delta z = 10\,\mathrm{m}$ was chosen to account for the turbulent exchange processes near surface and in the entrainment zone and also to guarantee a certain numerical stability. In a later step the vertical spacing was increased to check the sensitivity of vertical gradients to the grid spacing on the simulation results. <br>
The initial potential temperature profile is defined over a surface temperature of $\theta_\mathrm{surf.} = 290\,\mathrm{K}$ and constantly increases with height with a lapse rate of $\Gamma=0.006\,\mathrm{K\,m}^{-1}$. The initial wind is set to zero for all heights, turbulence is triggered by the addition of random perturbations to the temperature field in the lower atmosphere (up to $300\,$m). <br>
<br>
Starting from this two different types of simulations were performed: a Dry Convective Boundary Layer (DCBL) and a Cloud Topped Boundary Layer (CTBL). The DCBL is driven by a fixed sensible heat flux of $\overline{w'\theta'}_\mathrm{surf.}=0,1\,\mathrm{Kms^{-1}}$ and is integrated over $t_\mathrm{sim}= 3\,\mathrm{h}$. <br>
Since the CTBL case includes moisture it is possible to simulate nonprecipitating clouds that come with stongly non-linear processes. The moisture profile is initialized with an analytic profile:
\begin{equation}
q_\mathrm{v}= 0.008 \exp(-z\,/\,1500) \text{   .   } 
\label{eq:moisture}
\tag{26}
\end{equation}
With the addition of moisture the drive of the simulation is by flexible sensible and latent heat fluxes that result in a combined constant buoyancy flux
\begin{equation}
B = \frac{g}{\Theta}\,\overline{w'\theta'}_\mathrm{surf.} = 0.0007\,\mathrm{m}^2\mathrm{s}^{-3} \text{   ,   }
\label{eq:buoyflux}
\tag{27}
\end{equation}
with a reference temperature of $\Theta=290\,\mathrm{K}$. As integration time for this case is $t_\mathrm{sim}= 25\,\mathrm{h}$ and was chosen to account for the time span needed for the evolution of clouds.

## 5. Results
***

In a first step an analysis of temporal averaged horizontal cross sections will be shown. The reason for this is twofold: one they show if the horizontal SGS-TKE distribution is physically correct and second cross sections show eventual numeric artifacts when horizontal derivatives are not treated properly (e.g. parallelization techniques do not exchange necessary border grid points). Figure 2 shows $x$-$y$ cross-sections of the SGS-TKE at the second model level above the surface ($z=20\,\mathrm{m}$) evolving in time, temporally averaged over 15 minutes. Lower grid points show the common LES behaviour where vertical motions are underrepresented due to the surface limiting their diameter. After a spin-up of minutes a pattern typical for convective boundary layers can be observed. The appearing hexagonal cells can normaly be seen in temperature and vertical velocity cross-sections as a testament to the up- and downdraft distribution near the ground. Close to the ground a similar pattern in the SGS-TKE makes sense since its production is dominated by the production through buoyancy. The maxima of the SGS-TKE fall together with the updraft regions while minima are associated with downdrafts (not explicitly shown). While the physical behavior of the Deardorff scheme seems to be comprehensible, the smooth field over the whole domain and at the borders points out no signs of numerical problems with the domain decomposition on parallel architectures.

The second step will be to take a deeper look on the statistics of the DCBL by analyzing the vertical profiles of virtual potential temperature $\theta$, the turbulent vertical sensible heat flux $\overline{w'\,\theta'}$, the SGS-TKE $e$, and directly linked to that the eddy diffusivity $K_\mathrm{m}$.

## 6. Performance
***

Since the Deardorff scheme takes into account for the full prognostic equation of SGS-TKE more calculations have to be carried out. This directly results in a computational overhead that makes simulations more costly than those using the simpler Smagorinsky scheme. In this chapter, the run time of the different model parts will be analyzed quantitativly and the resulting overhead will be qualitativly assigned to their origin. It will be shown that there are differences with respect to the examined case and how these results could compare to real case studies. <br>
<br>
Table 1 shows the total run time and seperate ones for the model parts that differ the most between the Smagorinsky and the Deardorff scheme in the DCBL case, normalized by the total run time of the Smagorinsky scheme runtime. The run was carried out on processing elements (PEs).

| Model part | $\hat{t}_\mathrm{Smagorinsky}$  [s] | share of run time | $\hat{t}_\mathrm{Deardorff}$ [s]   | share of run time | relative difference |
|:----------:|:-----------------------------------:|:-----------------:|:----------------------------------:|:-----------------:|:-------------------:|
| turbulence |                               380.9 |              15.9 |                              467.4 |              17.1 |                22.6 |
| transport  |                                17.5 |               4.6 |                              263.7 |               9.6 |              1402.9 |
| total      |                                2400 |               100 |                             2734.6 |               100 |               13.72 |

*Table 1: Normalized run time for different model parts in DCBL case *

| Model part | $\hat{t}_\mathrm{Smagorinsky}$  [s] | share of run time | $\hat{t}_\mathrm{Deardorff}$ [s]   | share of run time | relative difference |
|:----------:|:-----------------------------------:|:-----------------:|:----------------------------------:|:-----------------:|:-------------------:|
| turbulence |                                 0.9 |              15.9 |                              467.4 |              17.1 |                22.6 |
| transport  |                                 0.5 |               4.6 |                              263.7 |               9.6 |              1402.9 |
| total      |                                2400 |               100 |                             2734.6 |               100 |               13.72 |

*Table 2: Run time for different model parts in CTBL case *

## 7. Conclusion / Summary
***

A Deardorff based prognostic equation for the SGS-TKE was implemented into the ICON-LEM as an additional turbulence closure scheme. The performance of the scheme was tested on two simple and idealized test cases and compared to another large-eddy model with an already implemented and validated Deardorff scheme.